---
Imports
---

In [2]:
import tensorflow as tf

---
Computational graph
Tensorflow core consists of 2 parts:
1. Building computational graph
2. Running computational graph

A computational graph is a series of TensorFlow operations arranged into a graph of nodes

Each node takes zero or more tensors as inputs and produces a tensor as an output. One type of node is a constant. Like all TensorFlow constants, it takes no inputs, and it outputs a value it stores internally. We can create two floating point Tensors node1 and node2 as follows:



---

In [3]:
node1 = tf.constant(3.0, dtype=tf.float32)
node2 = tf.constant(4.0) # dtype= tf.float32 by default
print(node1, node2)

Tensor("Const:0", shape=(), dtype=float32) Tensor("Const_1:0", shape=(), dtype=float32)


---

To actually evaluate the nodes, we must run the computational graph within a session. A session encapsulates the control and state of the TensorFlow runtime.

The following code creates a Session object and then invokes its run method to run enough of the computational graph to evaluate node1 and node2. By running the computational graph in a session as follows:

---

In [4]:
session = tf.Session()
print(session.run([node1, node2]))

[3.0, 4.0]


---

We can build more complicated computations by combining Tensor nodes with operations (Operations are also nodes). For example, we can add our two constant nodes and produce a new graph as follows:

---

In [8]:
from __future__ import print_function
node3 = tf.add(node1, node2)
print("node3: ", node3)
print("session.run(node3): ", session.run(node3))

node3:  Tensor("Add:0", shape=(), dtype=float32)
session.run(node3):  7.0


In [11]:

# A graph can be parameterized to accept external inputs, known as placeholders. A placeholder is a promise to provide a 
# value later.
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
c = a + b # same as tf.add(a, b)
print(session.run(c, {a : 3.5, b : 4.5}))
print(session.run(c, {a : [1, 3], b : [5, 8]}))

d = c * 3
print(session.run(d, {a : [1, 3], b : [5, 8]}))


8.0
[  6.  11.]
[ 18.  33.]


In [29]:
# Creating and initializing variables

W = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)
x = tf.placeholder(tf.float32)
linear_model = W*x + b

# variables must explicitly be innitialized with call to run on global_variables_initializer from session
init = tf.global_variables_initializer()
session.run(init)

# Can pass several values of x
print(session.run(linear_model, {x : [1, 2, 3, 4]}))

[ 0.          0.30000001  0.60000002  0.90000004]


In [30]:
# Evaluation the model on data:
y = tf.placeholder(tf.float32)
squared_deltas = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_deltas)
print(session.run(loss, {x : [1, 2, 3, 4], y : [0, -1, -2, -3]}))

# Reassigning values of variables:
fixW = tf.assign(W, [-1.])
fixb = tf.assign(b, [1.])
session.run([fixW, fixb])
print(session.run(loss, {x : [1, 2, 3, 4], y : [0, -1, -2, -3]}))

23.66
0.0


In [31]:
#Training 
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)
session.run(init) # Reset values to defaults
for i in range(1000):
    session.run(train, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]})
print(session.run([W, b]))

[array([-0.9999969], dtype=float32), array([ 0.99999082], dtype=float32)]


In [32]:
# Complate program for training a simple linear regression model:
import tensorflow as tf

# Model parameters
W = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)
# Model input and output
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)
linear_model = W*x + b


# loss
loss = tf.reduce_sum(tf.square(linear_model - y)) # sum of the squares
# optimizer
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

# training data
x_train = [1, 2, 3, 4]
y_train = [0, -1, -2, -3]
# training loop
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init) # reset values to wrong
for i in range(1000):
  sess.run(train, {x: x_train, y: y_train})

# evaluate training accuracy
curr_W, curr_b, curr_loss = sess.run([W, b, loss], {x: x_train, y: y_train})
print("W: %s b: %s loss: %s"%(curr_W, curr_b, curr_loss))

W: [-0.9999969] b: [ 0.99999082] loss: 5.69997e-11


---
tf.estimator
tf.estimator is a high-level TensorFlow library that simplifies the mechanics of machine learning, including the following:

running training loops
running evaluation loops
managing data sets
tf.estimator defines many common models.

Basic usage
Notice how much simpler the linear regression program becomes with tf.estimator:

---

In [33]:
# NumPy is often used to load, manipulate and preprocess data.
import numpy as np
import tensorflow as tf

# Declare list of features. We only have one numeric feature. There are many
# other types of columns that are more complicated and useful.
feature_columns = [tf.feature_column.numeric_column("x", shape=[1])]

# An estimator is the front end to invoke training (fitting) and evaluation
# (inference). There are many predefined types like linear regression,
# linear classification, and many neural network classifiers and regressors.
# The following code provides an estimator that does linear regression.
estimator = tf.estimator.LinearRegressor(feature_columns=feature_columns)

# TensorFlow provides many helper methods to read and set up data sets.
# Here we use two data sets: one for training and one for evaluation
# We have to tell the function how many batches
# of data (num_epochs) we want and how big each batch should be.
x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval  = np.array([2., 5., 8., 1.])
y_eval  = np.array([-1.01, -4.1, -7, 0.])
input_fn       = tf.estimator.inputs.numpy_input_fn({"x": x_train}, y_train, batch_size=4, num_epochs=None, shuffle=True)
train_input_fn = tf.estimator.inputs.numpy_input_fn({"x": x_train}, y_train, batch_size=4, num_epochs=1000, shuffle=False)
eval_input_fn  = tf.estimator.inputs.numpy_input_fn({"x": x_eval},  y_eval,  batch_size=4, num_epochs=1000, shuffle=False)

# We can invoke 1000 training steps by invoking the  method and passing the
# training data set.
estimator.train(input_fn=input_fn, steps=1000)

# Here we evaluate how well our model did.
train_metrics = estimator.evaluate(input_fn=train_input_fn)
eval_metrics  = estimator.evaluate(input_fn=eval_input_fn)
print("train metrics: %r"% train_metrics)
print("eval metrics: %r"% eval_metrics)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\snorr\\AppData\\Local\\Temp\\tmpdnnxaqi5', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000020B1765D320>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\snorr\AppData\Local\Temp\tmpdnnxaqi5\model.ckpt.
INFO:tensorflow:loss = 27.0, step = 1
INFO:tensorflow:global_step/sec: 1090.4
INFO:tensorflow:loss = 0.32995, step = 101 (0.092 sec)
INFO:tensorflow:global_step/sec: 1522.03
INFO:tensorflow:loss = 0.0959176, step = 201 (0.066 sec)
INF

In [34]:
# Creating costum model using estimator:
import numpy as np
import tensorflow as tf

# Declare list of features, we only have one real-valued feature
def model_fn(features, labels, mode):
    # Build a linear model and predict values
    W = tf.get_variable("W", [1], dtype=tf.float64)
    b = tf.get_variable("b", [1], dtype=tf.float64)
    y = W*features['x'] + b
    # Loss sub-graph
    loss = tf.reduce_sum(tf.square(y - labels))
    # Training sub-graph
    global_step = tf.train.get_global_step()
    optimizer = tf.train.GradientDescentOptimizer(0.01)
    train = tf.group(optimizer.minimize(loss), tf.assign_add(global_step, 1))
  # EstimatorSpec connects subgraphs we built to the
  # appropriate functionality.
    return tf.estimator.EstimatorSpec(
      mode=mode,
      predictions=y,
      loss=loss,
      train_op=train)

estimator = tf.estimator.Estimator(model_fn=model_fn)
# define our data sets
x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7., 0.])
input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=None, shuffle=True)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=1000, shuffle=False)
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_eval}, y_eval, batch_size=4, num_epochs=1000, shuffle=False)

# train
estimator.train(input_fn=input_fn, steps=1000)
# Here we evaluate how well our model did.
train_metrics = estimator.evaluate(input_fn=train_input_fn)
eval_metrics = estimator.evaluate(input_fn=eval_input_fn)
print("train metrics: %r"% train_metrics)
print("eval metrics: %r"% eval_metrics)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\snorr\\AppData\\Local\\Temp\\tmp5lotdlnc', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000020B17985668>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\snorr\AppData\Local\Temp\tmp5lotdlnc\model.ckpt.
INFO:tensorflow:loss = 2.72549420593, step = 1
INFO:tensorflow:global_step/sec: 1077.26
INFO:tensorflow:loss = 0.131959258717, step = 101 (0.095 sec)
INFO:tensorflow:global_step/sec: 1312.3
INFO:tensorflow:loss = 0.00649495542667, ste